# Neo4j Utilities

## Reset Neo4j Database

Delete all nodes and relationships from the database, and remove indexes and constraints.

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE') or 'neo4j'
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

from graphdatascience import GraphDataScience

# Use Neo4j URI and credentials according to your setup
# NEO4J_URI could look similar to "bolt://my-server.neo4j.io:7687"
gds = GraphDataScience(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# Check the installed GDS version on the server
print(f"GDS version: {gds.version()}")
assert gds.version()

### Reset Neo4j Database



In [ ]:
for _, constraint in gds.run_cypher('SHOW CONSTRAINTS yield name').iterrows():
    print(f"Removing constraint {constraint['name']}:")
    gds.run_cypher(f"DROP CONSTRAINT `{constraint['name']}`")

for _, index in gds.run_cypher('SHOW INDEXES yield name').iterrows():
    print(f"Removing index {index['name']}:")
    gds.run_cypher(f"DROP INDEX `{index['name']}`")

print("All constraints and indexes removed.")

# Delete all nodes and relationships
gds.run_cypher("MATCH (n) DETACH DELETE n")

print("All nodes and relationships deleted.")

## Neo4j Vector Search Utilities

In [ ]:
gds.run_cypher("SHOW VECTOR INDEXES")

In [ ]:
# Show graph schema
gds.run_cypher("CALL apoc.meta.schema()")

In [ ]:
def neo4j_vector_search(question, index_name=None, top_k=1, provider="OpenAI"):
  """Search for similar nodes using any available Neo4j vector index"""

  vectorIndexes = gds.run_cypher('SHOW INDEXES yield name')['name'].tolist()

  if index_name is not None:
    vectorIndexes = list(filter(lambda x: x == index_name, vectorIndexes))

  if len(vectorIndexes) == 0:
    if index_name is not None:
      raise ValueError(f"No vector index found with name '{index_name}'")
    raise ValueError(f"No vector indexes found in the database")

  # Prepare a vector embedding query statement depending on provider
  embedding_cypher = "WITH [] as questionEmbedding"
  match provider:
    case "OpenAI":
      embedding_cypher = f"""
          WITH genai.vector.encode($question, "OpenAI", {{token: $openAiApiKey}}) AS questionEmbedding
      """

  for indexName in vectorIndexes:
    print(f"Vector search of '{indexName}' ...")
    response = gds.run_cypher(
      f"""
      {embedding_cypher}
      CALL db.index.vector.queryNodes('{indexName}', {top_k}, questionEmbedding) 
          YIELD node, score
      RETURN score, node.text AS text
      """,
      params={
        'question': question,
        'openAiApiKey': OPENAI_API_KEY
      }
    )
    print(response)


In [ ]:
neo4j_vector_search("Tell me about Horatio.")